# Recommender Agent Cleanup (optional)

This notebook will walk through deleting all of the resources created in this step-by-step guide to create a Recommender agent. 

Resources have to be deleted in a specific sequence to avoid dependency errors. In order, we will delete event trackers, recommenders and campaigns, solutions, filters, datasets, and the dataset group.

## Load variables saved in prior Notebook

At the end of Notebook 2 we saved some variables that we'll need in this CleanUp Notebook. The following cell will load those variables into this lab environment.

In [ ]:
%store -r

## Delete dataset group and resources

We are now ready to delete the active dataset group and its resources. The `delete_all_resources` function will delete all dependent resources within the dataset groups starting with the event tracker, recommenders and campaigns, then solutions, filters, datasetsand ending with the dataset group itself.

*Note: if you get an error while running the following cell, try running the cell again.*

In [3]:
import boto3
import time

personalize = boto3.client('personalize')
    
def delete_resource(resource_arn, resource_type):
    print(f'Deleting {resource_type}: {resource_arn}')
    if resource_type == 'recommender':
        personalize.delete_recommender(recommenderArn=resource_arn)
    elif resource_type == 'eventTracker':
        personalize.delete_event_tracker(eventTrackerArn=resource_arn)
    elif resource_type == 'campaign':
        personalize.delete_campaign(campaignArn=resource_arn)
    elif resource_type == 'solution':
        personalize.delete_solution(solutionArn=resource_arn)
    elif resource_type == 'filter':
        personalize.delete_filter(filterArn=resource_arn)
    elif resource_type == 'dataset':
        personalize.delete_dataset(datasetArn=resource_arn)
    elif resource_type == 'schema':
        personalize.delete_schema(schemaArn=resource_arn)
    elif resource_type == 'datasetGroup':
        personalize.delete_dataset_group(datasetGroupArn=resource_arn)

# Check if there are any resources available for each type
def is_resource_list_empty(resource_list, resource_type):
    if resource_list is not None and len(resource_list) > 0:
        return False
    print(f'No {resource_type} resources found.')
    return True

# Check if there are any resources available and delete them if there are
def delete_all_resources():
    # Check if there are any Event Trackers available
    response = personalize.list_event_trackers(datasetGroupArn=dataset_group_arn)
    if not is_resource_list_empty(response['eventTrackers'], 'eventTracker'):
        for event_tracker in response['eventTrackers']:
            event_tracker_arn = event_tracker['eventTrackerArn']
            while True:
                try:
                    response = personalize.describe_event_tracker(
                        eventTrackerArn=event_tracker_arn
                    )
                except:
                    print('Recommender deleted successfully.')
                    break                
                status = response['eventTracker']['status']
                if status == 'PENDING' or status == "DELETE IN_PROGRESS":
                    print('Event Tracker deletion in progress...')
                elif status == 'ACTIVE':
                    delete_resource(event_tracker_arn, 'eventTracker')
                    print('Deleting Event Tracker...')
                else:
                    print('Event Tracker deleted successfully.')
                    break
                time.sleep(10)  # Wait for 10 seconds before checking again

    time.sleep(10)
    # Check if there are any Recommenders available
    response = personalize.list_recommenders(datasetGroupArn=dataset_group_arn)
    if not is_resource_list_empty(response['recommenders'], 'recommender'):
        for recommender in response['recommenders']:
            recommender_arn = recommender['recommenderArn']
            while True:
                # Check if there are any Recommenders available
                response = personalize.list_recommenders(datasetGroupArn=dataset_group_arn)
                if not is_resource_list_empty(response['recommenders'], 'recommender'):
                    try:
                        response = personalize.describe_recommender(
                            recommenderArn=recommender_arn
                        )
                    except:
                        print('Recommender deleted successfully.')
                        break
                    status = response['recommender']['status']
                    if status == 'PENDING' or status == "DELETE IN_PROGRESS":
                        print('Recommender deletion in progress...')
                    elif status == 'ACTIVE':
                        delete_resource(recommender_arn, 'recommender')
                        print('Deleting Recommender...')
                    else:
                        print('Recommender deleted successfully.')
                    time.sleep(10)  # Wait for 10 seconds before checking again
                else:
                    print('Recommender deleted successfully.')
                    break
                    
    time.sleep(10)
    # Check if there are any Campaigns available
    response = personalize.list_campaigns(solutionArn=similar_items_solution_version_arn)
    if not is_resource_list_empty(response['campaigns'], 'campaign'):
        for campaign in response['campaigns']:
            campaign_arn = campaign['campaignArn']
            while True:
                # Check if there are any Campaigns available
                response = personalize.list_campaigns(solutionArn=similar_items_solution_version_arn)
                if not is_resource_list_empty(response['campaigns'], 'campaign'):
                    try:
                        response = personalize.describe_campaign(
                            campaignArn=campaign_arn
                        )
                    except:
                        print('Campaign deleted successfully.')
                        break
                    status = response['campaign']['status']
                    if status == 'PENDING' or status == "DELETE IN_PROGRESS":
                        print('Campaign deletion in progress...')
                    elif status == 'ACTIVE':
                        delete_resource(campaign_arn, 'campaign')
                        print('Deleting Campaign...')
                    else:
                        print('Campaign deleted successfully.')
                    time.sleep(10)  # Wait for 10 seconds before checking again
                else:
                    print('Campaign deleted successfully.')
                    break

    time.sleep(10)
    # Check if there are any Solutions available
    response = personalize.list_solutions(datasetGroupArn=dataset_group_arn)
    if not is_resource_list_empty(response['solutions'], 'solution'):
        for solution in response['solutions']:
            solution_arn = solution['solutionArn']
            while True:
                # Check if there are any Solutions available
                response = personalize.list_solutions(datasetGroupArn=dataset_group_arn)
                if not is_resource_list_empty(response['solutions'], 'solution'):
                    try:
                        response = personalize.describe_solution(
                        solutionArn=solution_arn
                        )
                    except:
                        print('Solution deleted successfully.')
                        break
                    status = response['solution']['status']
                    if status == 'PENDING' or status == "DELETE IN_PROGRESS":
                        print('Solution deletion in progress...')
                    elif status == 'ACTIVE':
                        delete_resource(solution_arn, 'solution')
                        print('Deleting Solution...')
                    else:
                        print('Solution deleted successfully.')
                    time.sleep(10)  # Wait for 10 seconds before checking again
                else:
                    print('Solution deleted successfully.')
                    break
                
    time.sleep(10)
    # Check if there are any Filters available
    response = personalize.list_filters(datasetGroupArn=dataset_group_arn)
    if not is_resource_list_empty(response['Filters'], 'filter'):
        for filter in response['Filters']:
            while True:
                # Check if there are any Filters available
                response = personalize.list_filters(datasetGroupArn=dataset_group_arn)
                if not is_resource_list_empty(response['Filters'], 'filter'):
                    try:
                        response = personalize.describe_filter(
                            filterArn=filter['filterArn']
                        )
                    except:
                        print('Filter deleted successfully.')
                        break
                    status = response['filter']['status']
                    if status == 'PENDING' or status == "DELETE IN_PROGRESS":
                        print('Filter deletion in progress...')
                    elif status == 'ACTIVE':
                        delete_resource(filter['filterArn'], 'filter')
                        print('Deleting Filter...')
                    else:
                        print('Filter deleted successfully.')
                    time.sleep(10)  # Wait for 10 seconds before checking again  
                else:
                    print('Filter deleted successfully.')
                    break
                        
    time.sleep(10)
    # Check if there are any Datasets available
    response = personalize.list_datasets(datasetGroupArn=dataset_group_arn)
    if not is_resource_list_empty(response['datasets'], 'dataset'):
        for dataset in response['datasets']:
            while True:
                # Check if there are any Datasets available
                response = personalize.list_datasets(datasetGroupArn=dataset_group_arn)
                if not is_resource_list_empty(response['datasets'], 'dataset'):
                    try:
                        response = personalize.describe_dataset(
                            datasetArn=dataset['datasetArn']
                        )
                    except:
                        print('Dataset deleted successfully.')
                        break
                    status = response['dataset']['status']
                    if status == 'PENDING' or status == "DELETE IN_PROGRESS":
                        print('Dataset deletion in progress...')
                    elif status == 'ACTIVE':
                        delete_resource(dataset['datasetArn'], 'dataset')
                        print('Deleting Dataset Group...')
                    else:
                        print('Dataset deleted successfully.')
                    time.sleep(10)  # Wait for 10 seconds before checking again
                else:
                    print('Dataset deleted successfully.')
                    break 

    time.sleep(10)
    # Check if the Interactions Schema exist
    response = personalize.list_schemas()
    if not is_resource_list_empty(response['schemas'], 'schema'):
        for schema in response['schemas']:
            if schema['schemaArn']==interactions_schema_arn:
                delete_resource(interactions_schema_arn, 'schema')
                time.sleep(10)  # Wait for 10 seconds before checking again
                print('Schema deleted successfully.')
            else:
                print(f'No schema resource found.')
                break

    time.sleep(10)
    # Check if the Items Schema exist
    response = personalize.list_schemas()
    if not is_resource_list_empty(response['schemas'], 'schema'):
        for schema in response['schemas']:
            if schema['schemaArn']==items_schema_arn:
                delete_resource(items_schema_arn, 'schema')
                time.sleep(10)  # Wait for 10 seconds before checking again
                print('Schema deleted successfully.')
            else:
                print(f'No schema resource found.')
                break
                
    time.sleep(10)
    # Check if there are any Dataset Groups available
    response = personalize.list_dataset_groups()
    if not is_resource_list_empty(response['datasetGroups'], 'datasetGroup'):
        for dataset_group in response['datasetGroups']:
            if dataset_group['datasetGroupArn']==dataset_group_arn:
                while True:
                    try:
                        response = personalize.describe_dataset_group(
                            datasetGroupArn=dataset_group_arn
                        )
                    except:
                        print('Dataset Group deleted successfully.')
                        break
                    status = response['datasetGroup']['status']
                    if status == 'PENDING' or status == "DELETE IN_PROGRESS":
                        print('Dataset Group deletion in progress...')
                    elif status == 'ACTIVE':
                        delete_resource(dataset_group_arn, 'datasetGroup')
                        print('Deleting Dataset Group...')
                    else:
                        print('Dataset Group deleted successfully.')
                    time.sleep(10)  # Wait for 10 seconds before checking again
            else:
                print(f'No datasetGroup resources found.')
                break

delete_all_resources()


Deleting eventTracker: arn:aws:personalize:us-east-1:776299153297:event-tracker/14578154
Deleting Event Tracker...
Event Tracker deleted successfully.
Deleting recommender: arn:aws:personalize:us-east-1:776299153297:recommender/recommended_for_you_demo
Deleting Recommender...
Recommender deletion in progress...
Recommender deletion in progress...
Recommender deletion in progress...
Recommender deletion in progress...
Recommender deletion in progress...
Recommender deletion in progress...
Recommender deletion in progress...
Recommender deletion in progress...
Recommender deletion in progress...
Recommender deletion in progress...
Recommender deletion in progress...
Recommender deletion in progress...
Recommender deletion in progress...
Recommender deletion in progress...
Recommender deletion in progress...
Recommender deletion in progress...
Recommender deletion in progress...
Recommender deletion in progress...
Recommender deletion in progress...
Recommender deletion in progress...
Rec

## Delete Cloud Formation Stack

Now you can navigate to your [Cloudformation](https://console.aws.amazon.com/cloudformation/) console and delete the stack you created.

## Cleanup Complete

All resources created in the step-by-step guide have been deleted.